# 📈 Bitcoin Price Data Exploration

This notebook performs exploratory data analysis (EDA) on Bitcoin price data, including:
- Line plot of historical prices
- Seasonal decomposition
- Prophet-based forecasting
- Anomaly detection
- Optional interactive charts with Plotly

In [ ]:
# Simulated Example Data (for testing only)
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose

# Simulated Data
dates = pd.date_range(start='2023-01-01', periods=100, freq='H')
prices = 20000 + (pd.Series(range(100)) * 5) + pd.Series(range(100)).apply(lambda x: (x % 10 - 5) * 50)
df = pd.DataFrame({'timestamp': dates, 'price': prices})
df.set_index('timestamp', inplace=True)
df.head()

In [ ]:
# Load Real Data from S3 if available
from processing.preprocess import load_all_btc_data
df = load_all_btc_data()
df.head()

In [ ]:
# Plot Raw Bitcoin Prices
plt.figure(figsize=(10,4))
df['price'].plot(title='Bitcoin Price Over Time')
plt.xlabel('Timestamp')
plt.ylabel('USD')
plt.grid(True)
plt.show()

In [ ]:
# Seasonal Decomposition (24-hour pattern)
decomposition = seasonal_decompose(df['price'], model='additive', period=24)
decomposition.plot();

In [ ]:
# Prophet Forecasting
from prophet import Prophet
prophet_df = df.reset_index().rename(columns={'timestamp': 'ds', 'price': 'y'})
model = Prophet(daily_seasonality=True)
model.fit(prophet_df)
future = model.make_future_dataframe(periods=48, freq='H')
forecast = model.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
# Plot Forecast
fig1 = model.plot(forecast)

In [ ]:
# Anomaly Detection (Simple Z-Score Based)
import numpy as np
mean = df['price'].mean()
std = df['price'].std()
df['zscore'] = (df['price'] - mean) / std
df['anomaly'] = df['zscore'].abs() > 2
df[df['anomaly']].head()

In [ ]:
# Plot Anomalies
plt.figure(figsize=(10,4))
plt.plot(df.index, df['price'], label='Price')
plt.scatter(df.index[df['anomaly']], df['price'][df['anomaly']], color='red', label='Anomaly')
plt.title('Anomaly Detection')
plt.xlabel('Time')
plt.ylabel('Price (USD)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:

# Optional: Plotly Interactive Forecast
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

fig = go.Figure()
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], mode='lines', name='Forecast'))
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat_upper'], mode='lines', name='Upper Bound', line=dict(dash='dot')))
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat_lower'], mode='lines', name='Lower Bound', line=dict(dash='dot'),
                         fill='tonexty', fillcolor='rgba(173,216,230,0.2)'))

fig.update_layout(title='📈 Forecasted Bitcoin Prices (Plotly)',
                  xaxis_title='Time',
                  yaxis_title='Predicted Price (USD)',
                  template='plotly_white')

iplot(fig)